In [1]:
import os
# 【硬性要求】加载环境变量 (虽然单纯调用维基百科 API 不需要 OpenAI Key，
# 但作为端到端科研项目，保持此习惯可确保后续接入 LLM 时不出错)
from dotenv import load_dotenv
load_dotenv(override=True)

# 导入维基百科相关的工具和包装器
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# ==========================================
# 1. 配置并实例化工具 (Tool Configuration)
# ==========================================
# 步骤 A: 创建 API 包装器，设置检索策略
# - top_k_results=1: 只返回最顶端、最相关的那 1 个词条结果
# - doc_content_chars_max=1000: 截断文本，确保不超过 LLM 的处理极限
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)

# 步骤 B: 将包装器转化为标准的 LangChain 工具
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

# ==========================================
# 2. 探查工具的“自我描述” (Metadata Introspection)
# ==========================================
# 这些元数据对于未来构建 Agent 是极其重要的
print("=== 🛠️ 探查工具元数据 ===")
print(f"名称 (Name): {tool.name}")
print(f"描述 (Description): {tool.description}")
print(f"参数架构 (Args schema): {tool.args}\n")

# ==========================================
# 3. 执行工具 (Execution)
# ==========================================
if __name__ == "__main__":
    query_term = "Neural Network" # 老师演示中搜索“神经网络”
    print(f"=== 🔍 正在维基百科搜索: '{query_term}' ===")
    
    # 【科研级保姆注释】
    # 旧版本是：result = tool.run({"query": query_term})
    # 现代写法：使用 .invoke()。因为在最新的 LangChain 架构中，Tool 也是一个标准的 Runnable，
    # 使用 invoke 可以保持所有组件在 LCEL 调用风格上的一致性，且支持异步和流式输出。
    try:
        result = tool.invoke({"query": query_term})
        print("\n=== 📄 搜索结果 ===")
        print(result) # 预期打印出关于神经网络的维基摘要文本
    except Exception as e:
        print(f"\n❌ 执行失败，请检查网络连接或包安装: {e}")

=== 🛠️ 探查工具元数据 ===
名称 (Name): wikipedia
描述 (Description): A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
参数架构 (Args schema): {'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}

=== 🔍 正在维基百科搜索: 'Neural Network' ===

=== 📄 搜索结果 ===
Page: Neural network
Summary: A neural network is a group of interconnected units called neurons that send signals to one another. Neurons can be either biological cells or mathematical models. While individual neurons are simple, many of them together in a network can perform complex tasks. There are two main types of neural networks.

In neuroscience, a biological neural network is a physical structure found in brains and complex nervous systems – a population of nerve cells connected by synapses.
In machine learning, an artificial neural network is a mathematical model us